In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import copy
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os
import plotly.express as px
from matplotlib import cm
import matplotlib as mpl
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
import matplotlib as mpl
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy
import random
import sklearn
from sklearn.linear_model import LinearRegression
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn import tree
from sklearn.model_selection import cross_validate
import geopandas as gpd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pathos.threading import ThreadPool as Pool
from sklearn import datasets, linear_model
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [16]:
def r2_adj(y,y_pred,features):
    r2 = r2_score(y, y_pred)
    r2_adj = 1-(((1-r2)*(len(y)-1))/(len(y)-len(features)-1))
    return r2_adj

In [17]:
work_dir=Path("/home/fvanoorschot/")

In [18]:
data_sources = 'gswp-p_gleam-ep_gswp-t'

In [72]:
## nested cross validation
df = pd.read_csv(f'{work_dir}/master_characteristics_{data_sources}.csv',index_col=0)
df = df.drop(columns=['ppd_l'])
df = df[df.p_mean-df.q_mean < df.ep_mean]
df = df.dropna()
# features = ['t_mean','p_mean','idu_mean','asi','slp_mean']
features = ['t_mean','p_mean','ppd','cvp']
sr_rp = 'sr_20'
f = features 
dfx = df[f].values
dfy = df[sr_rp].values

# here split dataset into 'temp' and 'test'
cv_outer = 5 # the number of outer cross validations
sdf = pd.DataFrame(index=range(cv_outer), columns=['max_depth','min_samples_leaf','train_mae','train_r2_adj','test_mae','test_r2_adj','n_train','n_test'])
l = []
for i in range(cv_outer):
    X, X_test, y, y_test = train_test_split(dfx, dfy.reshape(-1, 1).ravel(), test_size=0.33)

    # run 'inner' cross validation'
    p_grid = {'max_depth':[5,6,7,8],'min_samples_leaf':np.arange(1,21,1)} # define paramters for grid search - add here also max features and change ranges
    cv_inner = 3 # the number of inner cross validations
    model = GridSearchCV(estimator=RandomForestRegressor(criterion='absolute_error',random_state=42),
                                                         param_grid=p_grid, cv=cv_inner, scoring='neg_mean_absolute_error',n_jobs=7)
    f = model.fit(X,y)
    df = pd.DataFrame(f.cv_results_)
    df = df.drop(columns=['mean_fit_time','std_fit_time','mean_score_time','std_score_time','params'])
    l.append(df) # store the output of inner cross validation in list

    # select the best model from inner cross validation
    d = df[df.rank_test_score==1]
    max_depth = d.param_max_depth.values[0]
    min_samples_leaf = d.param_min_samples_leaf.values[0]

    # run rf model with full 'temp' set and test set using the best inner cross validation
    rf = RandomForestRegressor(n_estimators=100,criterion='absolute_error',random_state=42,min_samples_leaf=min_samples_leaf,max_depth=max_depth,oob_score=False)
    rf.fit(X,y)
    test_y_pred = rf.predict(X_test)
    train_y_pred = rf.predict(X)
    n_train,n_test = len(y),len(y_test)

    train_mae = np.round(mean_absolute_error(y, train_y_pred),2)
    train_r2_adj =  np.round(r2_adj(y, train_y_pred,features),2)

    test_mae = np.round(mean_absolute_error(y_test, test_y_pred),2)
    test_r2_adj = np.round(r2_adj(y_test, test_y_pred,features),2)

    # save scores in dataframe
    sdf.loc[i][['max_depth','min_samples_leaf']] = [max_depth,min_samples_leaf]
    sdf.loc[i][['train_mae','train_r2_adj']] = [train_mae,train_r2_adj]
    sdf.loc[i][['test_mae','test_r2_adj']] = [test_mae,test_r2_adj]
    sdf.loc[i][['n_train','n_test']] = [n_train,n_test]

In [73]:
sdf

,max_depth,min_samples_leaf,train_mae,train_r2_adj,test_mae,test_r2_adj,n_train,n_test
0,8,1,33.1,0.86,44.64,0.74,2139,1055
1,8,1,32.68,0.86,45.48,0.76,2139,1055
2,8,2,33.86,0.85,44.22,0.73,2139,1055
3,8,1,33.61,0.86,42.16,0.77,2139,1055
4,8,1,33.08,0.86,45.22,0.76,2139,1055
